In [1]:
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
import pandas as pd
import time

In [2]:
for id in range(2021000980, 2021000981): #～から～まで繰り返し
    t1 = time.time()#処理前の時刻
    url = 'https://baseball.yahoo.co.jp/npb/game/' + str(id) + '/score?index=0110100'
    Base_url = 'https://baseball.yahoo.co.jp'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    res_check = res.status_code
    if res_check == 404 : #404のエラーの時
        print('該当urlなし')
        print(url)
        sleep(1)
    else: #問題なかった時
        print('情報取得中')
        print(url)
        sleep(1)
        #変数作り
        TotalP = 0
        Ball_Count = 0
        Strike_Count = 0
        Out_Count = 0
        Next_B_Index_L = 0
        Home_P = 0
        Away_P = 0
        Point_D = 0
        ALL_Datas =[]
        #試合情報まとめ
        Game_Info = soup.select('.bb-gameDescription')
        Game_Info_A = Game_Info[0].text.splitlines()
        #試合日、開始時間、球場
        Game_Day = Game_Info_A[1].lstrip()
        Game_Time = Game_Info_A[2].lstrip()
        Game_Stadium = Game_Info_A[3].lstrip()
        #アウェイチーム（先攻）
        away = soup.select('#gm_mema h1')
        away_team = away[0].text
        #ホームチーム（後攻）
        home = soup.select('#gm_memh h1')
        home_team = home[0].text
        #何回表裏、試合終了等情報
        Inning = soup.find(class_='live')
        Inning_data = Inning.em.text
        #ゲームID
        GameID_O = soup.find_all(class_='next')
        GameID_S = GameID_O[1].a.attrs['href']
        GameID_T = str(GameID_S)
        GameID_F = re.findall(r"\d+", GameID_T)
        GameID = GameID_F[0]

        #Inning_dataが終了を含まないとき繰り返し
        while '終了' not in Inning_data:
            #イニングインデックスの末尾が0の時繰り返し
            while Next_B_Index_L == 0:
                i = 0
                BatR = soup.find_all(class_='bb-splitsTable__data--result')
                INDEX = soup.find_all(id='result')
                INDEX_str = str(INDEX)
                RBI_O = soup.select('#result span')
                #初球、牽制アウトのとき
                if len(BatR) == 0:
                    if '盗塁失敗' in INDEX_str:
                        Out_Count += 1
                    if 'タッチアウト' in INDEX_str:
                        Out_Count += 1
                    FO = str(RBI_O)
                    if 'アウト' in FO:
                        Out_Count += 1

                #3アウト時
                if Out_Count == 3:
                    Out_Count = 0
                for i in range(0, len(BatR)):
                    TotalP += 1
                    #変数iを利用しない
                    #投手名
                    Pit = soup.select('#pit a')
                    Pit_Name = Pit[1].text
                    #打者名
                    Batt = soup.select('#batt a')
                    Batt_Name = Batt[1].text
                    #投手左右投
                    Pit_h = soup.select('#pit .dominantHand')
                    Pit_Hand = Pit_h[0].text
                    #打者左右打
                    Batt_h = soup.select('#batt .dominantHand')
                    Batt_Hand =Batt_h[0].text
                    #ランナーの有無
                    #１塁
                    base_1 = soup.select('#base1')
                    if len(base_1) == 0:
                        base1 = 0
                    else:
                        base1 = 1
                    #２塁
                    base_2 = soup.select('#base2')
                    if len(base_2) == 0:
                        base2 = 0
                    else:
                        base2 = 1
                    #３塁
                    base_3 = soup.select('#base3')
                    if len(base_3) == 0:
                        base3 = 0
                    else:
                        base3 = 1
                        
                    #点差
                    if '表' in Inning_data:
                        Point_D = Away_P - Home_P
                    else:
                        Point_D = Home_P - Away_P
                        
                    #打点
                    RBI_O = soup.select('#result span')
                    s_RBI = str(RBI_O)
                    if len(RBI_O) == 0:
                        RBI = 0
                    else:
                        RBI_S = RBI_O[0].text.split()
                        if len(RBI_S) != 2 or 'アウト' in s_RBI or 'セーフ' in s_RBI:
                            RBI = 0
                        else:
                            if i + 1 == len(BatR):
                                RBI_T = str(RBI_S)
                                RBI_f = re.findall(r"\d+", RBI_T)
                                if len(RBI_f) == 2:
                                    RBI = int(RBI_f[1])
                                else:
                                    RBI = int(RBI_f[0])
                            else:
                                RBI = 0
                   
                    #変数iを利用する
                    #打席内球数
                    atBatP = i + 1
                    #球種
                    BallType_Info = soup.find_all(class_='bb-splitsTable__data--ballType')
                    BallType = BallType_Info[i].text
                    #球種番号取得
                    if 'ストレート' in BallType:
                        BallT = 0
                    elif 'ツーシーム' in BallType:
                        BallT = 1
                    elif 'スライダー' in BallType or 'カットボール' in BallType:
                        BallT = 2
                    elif 'カーブ' in BallType:
                        BallT = 3
                    elif 'フォーク' in BallType or 'チェンジアップ' in BallType or 'スプリット' in BallType or 'ナックル' in BallType:
                        BallT = 4
                    elif 'シンカー' in BallType or 'スクリュー' in BallType:
                        BallT = 5
                    elif 'シュート' in BallType:
                        BallT = 6
                    else:
                        BallT = 99
                    #球速
                    BallS = soup.find_all(class_='bb-splitsTable__data--speed')
                    Ball_S = BallS[i].text
                    BallSpeed = re.sub(r"\D", "", Ball_S)
                    #投手の投球数
                    Pit_Result = soup.find_all(class_='bb-splits__item')
                    Pit_Total = Pit_Result[1].find_all(class_='bb-splitsTable__data')
                    Pit_TotalP = int(Pit_Total[(i*5)+6].text)
                    #打席内容
                    BatR = soup.find_all(class_='bb-splitsTable__data--result')
                    BatResult_O = BatR[i].text.strip()
                    BatResult_S = BatResult_O.replace('\n', '')
                    BatResult = BatResult_S.replace(' ', '')
                    #投球情報、前半コース、後半その打席の投球数
                    Ball_Info = soup.find_all(class_='bb-icon__ballCircle')
                    course = Ball_Info[i].attrs['style']
                    result = re.findall(r"\d+", course)
                    #コース高低
                    HighLow = int(result[0]) + 12
                    #コース内外
                    InOut = int(result[1]) + 12
                    #コース
                    if HighLow <= 40:
                        if InOut<=32:
                            Course_XY = 1
                        elif 32<InOut<=64:
                            Course_XY = 2
                        elif 64<InOut<=96:
                            Course_XY = 3
                        elif 96<InOut<=128:
                            Course_XY = 4
                        else:
                            Course_XY = 5

                    elif 40 < HighLow <= 80:
                        if InOut<=32:
                            Course_XY = 6
                        elif 32<InOut<=64:
                            Course_XY = 7
                        elif 64<InOut<=96:
                            Course_XY = 8
                        elif 96<InOut<=128:
                            Course_XY = 9
                        else:
                            Course_XY = 10

                    elif 80 < HighLow <= 120:
                        if InOut<=32:
                            Course_XY = 11
                        elif 32<InOut<=64:
                            Course_XY = 12
                        elif 64<InOut<=96:
                            Course_XY = 13
                        elif 96<InOut<=128:
                            Course_XY = 14
                        else:
                            Course_XY = 15

                    elif 120 < HighLow <= 160:
                        if InOut<=32:
                            Course_XY = 16
                        elif 32<InOut<=64:
                            Course_XY = 17
                        elif 64<InOut<=96:
                            Course_XY = 18
                        elif 96<InOut<=128:
                            Course_XY = 19
                        else:
                            Course_XY = 20

                    else:
                        if InOut<=32:
                            Course_XY = 21
                        elif 32<InOut<=64:
                            Course_XY = 22
                        elif 64<InOut<=96:
                            Course_XY = 23
                        elif 96<InOut<=128:
                            Course_XY = 24
                        else:
                            Course_XY = 25   
                    #スイング有無
                    if 'ボール' in BatResult or '見' in BatResult or '四球' in BatResult or '死球' in BatResult:
                        Swing = 0
                        Contact = 0
                    else:
                        if '空' in BatResult:
                            Swing = 1
                            Contact = 1
                        else:
                            Swing = 1
                            Contact = 2
                    #打球方向
                    if '一' in BatResult and '一歩' not in BatResult and '一塁' not in BatResult and '間一髪' not in BatResult:
                        BatD = 3
                    elif '二' in BatResult:
                        BatD = 4
                    elif '三' in BatResult and '三振' not in BatResult:
                        BatD = 5
                    elif '遊' in BatResult:
                        BatD = 6
                    elif '左' in BatResult:
                        BatD = 7
                    elif '右' in BatResult:
                        BatD = 9
                    elif '中' in BatResult:
                        BatD = 8
                    elif '一安[一塁ベース直撃]' in BatResult:
                        BatD = 3
                    elif '捕' in BatResult:
                        BatD = 2
                    elif '投' in BatResult and '[投手ガッツポーズ]' not in BatResult and '放り投げる' not in BatResult:
                        BatD = 1
                    else:
                        BatD = 0
                    #打球種類
                    if 'ゴロ' in BatResult:
                        BatT = 1
                    elif '飛' in BatResult and '犠飛' not in BatResult and '三振' not in BatResult and '飛ぶ' not in BatResult:
                        BatT = 2
                    elif '直' in BatResult and '直撃' not in BatResult:
                        BatT = 3
                    elif '犠打' in BatResult or '犠野' in BatResult:
                        BatT = 4
                    elif '犠飛' in BatResult:
                        BatT = 5
                    elif '三振' in BatResult:
                        BatT = 6
                    elif '安' in BatResult:
                        BatT = 7
                    elif '２' in BatResult:
                        BatT = 8
                    elif '３' in BatResult:
                        BatT = 9
                    elif '本' in BatResult:
                        BatT = 10
                    elif '四球' in BatResult:
                        BatT = 11
                    elif '死球' in BatResult:
                        BatT = 12
                    elif '失' in BatResult and 'バント失敗' not in BatResult:
                        BatT = 13
                    elif '併' in BatResult:
                        BatT = 14
                    else:
                        BatT = 0
                    #ストライクボール等カウント判定
                    Count_info = soup.select('.balllist tr')
                    Count = str(Count_info[i].attrs['class'])
                    INDEX = soup.find_all(id='result')
                    INDEX_str = str(INDEX)
                    if i + 1 == len(BatR):
                        if '申告敬遠' in RBI_O:
                            Count_R = 4
                        elif 'ball3' in Count and '併' in BatResult and '崩' not in BatResult and 'ミス' not in INDEX_str:
                            Count_R = 3
                        elif 'ball2' in Count and Ball_Count != 3:
                            Count_R = 0
                        elif 'ball2' in Count or 'ball4' in Count:
                            Count_R = 4
                        elif 'ball1' in Count and '振逃' in BatResult:
                            Count_R = 4
                        elif 'ball1' in Count and 'バ三振' in BatResult:
                            Count_R = 2
                        elif 'ball1' in Count and Strike_Count != 2:
                            Count_R = 1
                        elif 'ball1' in Count and 'ファウル' in BatResult and 'チップ' not in BatResult:
                            Count_R = 1
                        elif 'ball1' in Count or 'ball3' in Count or 'ball5' in Count:
                            Count_R = 2
                    else: 
                        if 'ball2' in Count:
                            Count_R = 0
                        else:
                            Count_R = 1
                            
                    #打順、守備位置
                    Bat_info = soup.select('.bb-splitsTable__row--live td')
                    Bat_Order = Bat_info[0].text
                    Bat_fp = Bat_info[1].text 

                    ALL_Data = {
                    '試合全体の投球数': TotalP,
                    '各投手の投球数': Pit_TotalP,
                    '打席投球数': atBatP,
                    '投手名': Pit_Name,
                    '左右投': Pit_Hand,
                    '打者名': Batt_Name,
                    '左右打': Batt_Hand,
                    '何回': Inning_data,
                    '打順': Bat_Order,
                    '打者守備位置': Bat_fp,
                    'B': Ball_Count,
                    'S': Strike_Count,
                    'O': Out_Count,
                    '1塁': base1,
                    '2塁': base2,
                    '3塁': base3,
                    '球種': BallType,
                    '球種番号': BallT,
                    '球速': BallSpeed,
                    'コース内外': InOut,
                    'コース高低': HighLow,
                    'コース': Course_XY,
                    'スイング': Swing,
                    'コンタクト': Contact,
                    '内容': BatResult,
                    '打球方向': BatD,
                    '打球種類': BatT,
                    '結果': Count_R,
                    '打点': RBI,
                    '先攻得点計': Away_P,
                    '後攻得点計': Home_P,
                    '点差': Point_D,
                    '試合日': Game_Day,
                    '開始時間': Game_Time,
                    '先攻（アウェイ）': away_team,
                    '後攻（ホーム）': home_team,
                    '球場': Game_Stadium,
                    'ゲームID': GameID,
                    }

                    ALL_Datas.append(ALL_Data)
                    
                    #得点計追加
                    if '表' in Inning_data:
                        Away_P += int(RBI)
                    else:
                        Home_P += int(RBI)

                    #カウント追加
                    if Count_R == 0:
                        Ball_Count += 1
                    elif Count_R == 1:
                        Strike_Count += 1
                        if Strike_Count == 3:
                            Strike_Count = 2
                    elif Count_R == 2:
                        Out_Count += 1
                        Ball_Count = 0
                        Strike_Count = 0
                    elif Count_R == 3:
                        Out_Count += 2
                        Ball_Count = 0
                        Strike_Count = 0
                    elif Count_R == 4:
                        Ball_Count = 0
                        Strike_Count = 0
                        
                    #盗塁失敗時等
                    if i + 1 == len(BatR):
                        #盗塁死
                        if '盗塁失敗' in INDEX_str:
                            Out_Count += 1
                        #捕手牽制アウト
                        if '捕手けん制' in INDEX_str and '盗塁成功' not in INDEX_str and '悪送球' not in INDEX_str:
                            Out_Count += 1
                        #安打かつ走者アウト
                        if 'タッチアウト' in INDEX_str and Count_R != 2:
                            Out_Count += 1
                        #犠牲フライ成功かつ、関係ないところでアウト
                        if 'タッチアウト' in INDEX_str and 'ball5' in Count:
                            Out_Count += 1
                        #犠牲フライ失敗等、フライ、ライナーによる併殺打
                        if 'ライナー' in INDEX_str or 'フライ' in INDEX_str:
                            if 'アウト' in INDEX_str:
                                Out_Count += 1
                        #牽制アウト
                        FO = str(RBI_O)
                        if 'アウト' in FO:
                            Out_Count += 1

                    #3アウト時
                    if Out_Count >= 3:
                        Out_Count = 0

                #イニングインデックスの末尾を取得
                Next_B = soup.find_all(class_='next')
                Next_B_Index = Next_B[1].a.attrs['index']
                Next_B_Index_L = int(Next_B_Index[-1])
                #次へのボタンのリンクを取得
                Next_B = soup.find_all(class_='next')
                Next_B_Link = Next_B[1].a.attrs['href']
                #ベースと結合、読み込み
                url = Base_url + Next_B_Link
                res = requests.get(url)
                soup = BeautifulSoup(res.text, 'html.parser')
                sleep(1)
                #何回表裏、試合終了等情報取得
                Inning = soup.find(class_='live')
                Inning_data = Inning.em.text

                #イニングデータに終了を含むとき、while抜け
                if '終了' in Inning_data:
                    break

            #イニングインデックスの末尾が0でないときの操作
            else:
                INDEX = soup.find_all(id='result')
                INDEX_str = str(INDEX)
                #INDEX_strが選手交代を示すとき
                if '【継投】' in INDEX_str or '【守備】' in INDEX_str or '【代打】' in INDEX_str:
                    #イニングインデックスの末尾を取得
                    Next_B = soup.find_all(class_='next')
                    Next_B_Index = Next_B[1].a.attrs['index']
                    Next_B_Index_L = int(Next_B_Index[-1])
                    #次へのボタンのリンクを取得
                    Next_B = soup.find_all(class_='next')
                    Next_B_Link = Next_B[1].a.attrs['href']
                    #ベースと結合、読み込み
                    url = Base_url + Next_B_Link
                    res = requests.get(url)
                    soup = BeautifulSoup(res.text, 'html.parser')
                    sleep(1)
                #INDEX_strが打席の続きのとき
                else:
                    if len(BatR) == 0:
                        i = -1
                    BatR = soup.find_all(class_='bb-splitsTable__data--result')
                    for i in range(i+1, len(BatR)):
                        TotalP += 1
                        #変数iを利用しない
                        #投手名
                        Pit = soup.select('#pit a')
                        Pit_Name = Pit[1].text
                        #打者名
                        Batt = soup.select('#batt a')
                        Batt_Name = Batt[1].text
                        #投手左右投
                        Pit_h = soup.select('#pit .dominantHand')
                        Pit_Hand = Pit_h[0].text
                        #打者左右打
                        Batt_h = soup.select('#batt .dominantHand')
                        Batt_Hand =Batt_h[0].text
                        #ランナーの有無
                        #１塁
                        base_1 = soup.select('#base1')
                        if len(base_1) == 0:
                            base1 = 0
                        else:
                            base1 = 1
                        #２塁
                        base_2 = soup.select('#base2')
                        if len(base_2) == 0:
                            base2 = 0
                        else:
                            base2 = 1
                        #３塁
                        base_3 = soup.select('#base3')
                        if len(base_3) == 0:
                            base3 = 0
                        else:
                            base3 = 1
                        
                        #点差
                        if '表' in Inning_data:
                            Point_D = Away_P - Home_P
                        else:
                            Point_D = Home_P - Away_P
                        
                        #打点
                        RBI_O = soup.select('#result span')
                        s_RBI = str(RBI_O)
                        if len(RBI_O) == 0:
                            RBI = 0
                        else:
                            RBI_S = RBI_O[0].text.split()
                            if len(RBI_S) != 2 or 'アウト' in s_RBI or 'セーフ' in s_RBI:
                                RBI = 0
                            else:
                                if i + 1 == len(BatR):
                                    RBI_T = str(RBI_S)
                                    RBI_f = re.findall(r"\d+", RBI_T)
                                    if len(RBI_f) == 2:
                                        RBI = int(RBI_f[1])
                                    else:
                                        RBI = int(RBI_f[0])
                                else:
                                    RBI = 0

                        #変数iを利用する
                        #打席内投球数
                        atBatP = i + 1
                        #球種
                        BallType_Info = soup.find_all(class_='bb-splitsTable__data--ballType')
                        BallType = BallType_Info[i].text
                        #球種番号取得
                        if 'ストレート' in BallType:
                            BallT = 0
                        elif 'ツーシーム' in BallType:
                            BallT = 1
                        elif 'スライダー' in BallType or 'カットボール' in BallType:
                            BallT = 2
                        elif 'カーブ' in BallType:
                            BallT = 3
                        elif 'フォーク' in BallType or 'チェンジアップ' in BallType or 'スプリット' in BallType or 'ナックル' in BallType:
                            BallT = 4
                        elif 'シンカー' in BallType or 'スクリュー' in BallType:
                            BallT = 5
                        elif 'シュート' in BallType:
                            BallT = 6
                        else:
                            BallT = 99
                        #球速
                        BallS = soup.find_all(class_='bb-splitsTable__data--speed')
                        Ball_S = BallS[i].text
                        BallSpeed = re.sub(r"\D", "", Ball_S)
                        #投手の投球数
                        Pit_Result = soup.find_all(class_='bb-splits__item')
                        Pit_Total = Pit_Result[1].find_all(class_='bb-splitsTable__data')
                        Pit_TotalP = int(Pit_Total[(i*5)+6].text)
                        #打席内容
                        BatR = soup.find_all(class_='bb-splitsTable__data--result')
                        BatResult_O = BatR[i].text.strip()
                        BatResult_S = BatResult_O.replace('\n', '')
                        BatResult = BatResult_S.replace(' ', '')
                        #投球情報、前半コース、後半その打席の投球数
                        Ball_Info = soup.find_all(class_='bb-icon__ballCircle')
                        course = Ball_Info[i].attrs['style']
                        result = re.findall(r"\d+", course)
                        #コース高低
                        HighLow = int(result[0]) + 12
                        #コース内外
                        InOut = int(result[1]) + 12
                        #コース
                        if HighLow <= 40:
                            if InOut<=32:
                                Course_XY = 1
                            elif 32<InOut<=64:
                                Course_XY = 2
                            elif 64<InOut<=96:
                                Course_XY = 3
                            elif 96<InOut<=128:
                                Course_XY = 4
                            else:
                                Course_XY = 5

                        elif 40 < HighLow <= 80:
                            if InOut<=32:
                                Course_XY = 6
                            elif 32<InOut<=64:
                                Course_XY = 7
                            elif 64<InOut<=96:
                                Course_XY = 8
                            elif 96<InOut<=128:
                                Course_XY = 9
                            else:
                                Course_XY = 10

                        elif 80 < HighLow <= 120:
                            if InOut<=32:
                                Course_XY = 11
                            elif 32<InOut<=64:
                                Course_XY = 12
                            elif 64<InOut<=96:
                                Course_XY = 13
                            elif 96<InOut<=128:
                                Course_XY = 14
                            else:
                                Course_XY = 15

                        elif 120 < HighLow <= 160:
                            if InOut<=32:
                                Course_XY = 16
                            elif 32<InOut<=64:
                                Course_XY = 17
                            elif 64<InOut<=96:
                                Course_XY = 18
                            elif 96<InOut<=128:
                                Course_XY = 19
                            else:
                                Course_XY = 20

                        else:
                            if InOut<=32:
                                Course_XY = 21
                            elif 32<InOut<=64:
                                Course_XY = 22
                            elif 64<InOut<=96:
                                Course_XY = 23
                            elif 96<InOut<=128:
                                Course_XY = 24
                            else:
                                Course_XY = 25   
                        #スイング有無
                        if 'ボール' in BatResult or '見' in BatResult or '四球' in BatResult or '死球' in BatResult:
                            Swing = 0
                            Contact = 0
                        else:
                            if '空' in BatResult:
                                Swing = 1
                                Contact = 1
                            else:
                                Swing = 1
                                Contact = 2
                        #打球方向
                        if '一' in BatResult and '一歩' not in BatResult and '一塁' not in BatResult and '間一髪' not in BatResult:
                            BatD = 3
                        elif '二' in BatResult:
                            BatD = 4
                        elif '三' in BatResult and '三振' not in BatResult:
                            BatD = 5
                        elif '遊' in BatResult:
                            BatD = 6
                        elif '左' in BatResult:
                            BatD = 7
                        elif '右' in BatResult:
                            BatD = 9
                        elif '中' in BatResult:
                            BatD = 8
                        elif '一安[一塁ベース直撃]' in BatResult:
                            BatD = 3
                        elif '捕' in BatResult:
                            BatD = 2
                        elif '投' in BatResult and '[投手ガッツポーズ]' not in BatResult and '放り投げる' not in BatResult:
                            BatD = 1
                        else:
                            BatD = 0
                        #打球種類
                        if 'ゴロ' in BatResult:
                            BatT = 1
                        elif '飛' in BatResult and '犠飛' not in BatResult and '三振' not in BatResult and '飛ぶ' not in BatResult:
                            BatT = 2
                        elif '直' in BatResult and '直撃' not in BatResult:
                            BatT = 3
                        elif '犠打' in BatResult or '犠野' in BatResult:
                            BatT = 4
                        elif '犠飛' in BatResult:
                            BatT = 5
                        elif '三振' in BatResult:
                            BatT = 6
                        elif '安' in BatResult:
                            BatT = 7
                        elif '２' in BatResult:
                            BatT = 8
                        elif '３' in BatResult:
                            BatT = 9
                        elif '本' in BatResult:
                            BatT = 10
                        elif '四球' in BatResult:
                            BatT = 11
                        elif '死球' in BatResult:
                            BatT = 12
                        elif '失' in BatResult and 'バント失敗' not in BatResult:
                            BatT = 13
                        elif '併' in BatResult:
                            BatT = 14
                        else:
                            BatT = 0
                        #ストライクボール等カウント判定
                        Count_info = soup.select('.balllist tr')
                        Count = str(Count_info[i].attrs['class'])
                        INDEX = soup.find_all(id='result')
                        INDEX_str = str(INDEX)
                        if i + 1 == len(BatR):
                            if '申告敬遠' in RBI_O:
                                Count_R = 4
                            elif 'ball3' in Count and '併' in BatResult and '崩' not in BatResult and 'ミス' not in INDEX_str:
                                Count_R = 3
                            elif 'ball2' in Count and Ball_Count != 3:
                                Count_R = 0
                            elif 'ball2' in Count or 'ball4' in Count:
                                Count_R = 4
                            elif 'ball1' in Count and '振逃' in BatResult:
                                Count_R = 4
                            elif 'ball1' in Count and 'バ三振' in BatResult:
                                Count_R = 2
                            elif 'ball1' in Count and Strike_Count != 2:
                                Count_R = 1
                            elif 'ball1' in Count and 'ファウル' in BatResult and 'チップ' not in BatResult:
                                Count_R = 1
                            elif 'ball1' in Count or 'ball3' in Count or 'ball5' in Count:
                                Count_R = 2
                        else: 
                            if 'ball2' in Count:
                                Count_R = 0
                            else:
                                Count_R = 1
                                
                        #打順、守備位置
                        Bat_info = soup.select('.bb-splitsTable__row--live td')
                        Bat_Order = Bat_info[0].text
                        Bat_fp = Bat_info[1].text 
                        
                        ALL_Data = {
                        '試合全体の投球数': TotalP,
                        '各投手の投球数': Pit_TotalP,
                        '打席投球数': atBatP,
                        '投手名': Pit_Name,
                        '左右投': Pit_Hand,
                        '打者名': Batt_Name,
                        '左右打': Batt_Hand,
                        '何回': Inning_data,
                        '打順': Bat_Order,
                        '打者守備位置': Bat_fp,
                        'B': Ball_Count,
                        'S': Strike_Count,
                        'O': Out_Count,
                        '1塁': base1,
                        '2塁': base2,
                        '3塁': base3,
                        '球種': BallType,
                        '球種番号': BallT,
                        '球速': BallSpeed,
                        'コース内外': InOut,
                        'コース高低': HighLow,
                        'コース': Course_XY,
                        'スイング': Swing,
                        'コンタクト': Contact,
                        '内容': BatResult,
                        '打球方向': BatD,
                        '打球種類': BatT,
                        '結果': Count_R,
                        '打点': RBI,
                        '先攻得点計': Away_P,
                        '後攻得点計': Home_P,
                        '点差': Point_D,
                        '試合日': Game_Day,
                        '開始時間': Game_Time,
                        '先攻（アウェイ）': away_team,
                        '後攻（ホーム）': home_team,
                        '球場': Game_Stadium,
                        'ゲームID': GameID,
                        }

                        ALL_Datas.append(ALL_Data)
                        
                        #得点計追加
                        if '表' in Inning_data:
                            Away_P += int(RBI)
                        else:
                            Home_P += int(RBI)

                        #カウント追加
                        if Count_R == 0:
                            Ball_Count += 1
                        elif Count_R == 1:
                            Strike_Count += 1
                            if Strike_Count == 3:
                                Strike_Count = 2
                        elif Count_R == 2:
                            Out_Count += 1
                            Ball_Count = 0
                            Strike_Count = 0
                        elif Count_R == 3:
                            Out_Count += 2
                            Ball_Count = 0
                            Strike_Count = 0
                        elif Count_R == 4:
                            Ball_Count = 0
                            Strike_Count = 0
                            
                        #盗塁失敗時等
                        if i + 1 == len(BatR):
                            #盗塁死
                            if '盗塁失敗' in INDEX_str:
                                Out_Count += 1
                            #捕手牽制アウト
                            if '捕手けん制' in INDEX_str and '盗塁成功' not in INDEX_str and '悪送球' not in INDEX_str:
                                Out_Count += 1
                            #安打かつ走者アウト
                            if 'タッチアウト' in INDEX_str and Count_R != 2:
                                Out_Count += 1
                            #犠牲フライ成功かつ、関係ないところでアウト
                            if 'タッチアウト' in INDEX_str and 'ball5' in Count:
                                Out_Count += 1
                            #犠牲フライ失敗等、フライ、ライナーによる併殺打
                            if 'ライナー' in INDEX_str or 'フライ' in INDEX_str:
                                if 'アウト' in INDEX_str:
                                    Out_Count += 1
                            #牽制アウト
                            FO = str(RBI_O)
                            if 'アウト' in FO:
                                Out_Count += 1
                        
                        #3アウト時
                        if Out_Count >= 3:
                            Out_Count = 0

                    #イニングインデックスの末尾を取得
                    Next_B = soup.find_all(class_='next')
                    Next_B_Index = Next_B[1].a.attrs['index']
                    Next_B_Index_L = int(Next_B_Index[-1])
                    #次へのボタンのリンクを取得
                    Next_B = soup.find_all(class_='next')
                    Next_B_Link = Next_B[1].a.attrs['href']
                    #ベースと結合、読み込み
                    url = Base_url + Next_B_Link
                    res = requests.get(url)
                    soup = BeautifulSoup(res.text, 'html.parser')
                    sleep(1)

                    #何回表裏、試合終了等情報
                    Inning = soup.find(class_='live')
                    Inning_data = Inning.em.text


        #イニングデータが試合終了となったらエクセルファイル保存、保存完了とプリント  
        else:
            print(f"投球数:{TotalP},先攻得点計：{Away_P},後攻得点計：{Home_P},アウトカウント：{Out_Count},内容：{BatResult}")
            #正しい投球数
            url = 'https://baseball.yahoo.co.jp/npb/game/' + str(id) + '/stats'
            res = requests.get(url)
            soup = BeautifulSoup(res.text, 'html.parser')
            P_Result = soup.find_all(class_='bb-scoreTable__dataLabel')
            P_len = int(len(P_Result) / 12)
            P = 0
            for num in range(0,P_len):
                P += int(P_Result[2+(12*num)].text)
            #正しい点数
            Point = soup.find_all(class_='bb-teamScoreTable__data--score')
            A_P = Point[0].text
            H_P = Point[1].text
            print(f"正しい投球数:{P},先攻得点計：{A_P},後攻得点計：{H_P}")
            pd.DataFrame(ALL_Datas).to_excel('エクセルファイル/全試合元データ/' + GameID + '.xlsx', index = False)
            print('保存完了')
            # 処理後の時刻
            t2 = time.time()

            # 経過時間を表示
            elapsed_time = t2-t1
            print(f"経過時間：{elapsed_time}")

print('全データ完了')

情報取得中
https://baseball.yahoo.co.jp/npb/game/2021000980/score?index=0110100
投球数:398,先攻得点計：2,後攻得点計：1,アウトカウント：0,内容：二ゴロ[バウンド高い、観客大歓声]
正しい投球数:398,先攻得点計：2,後攻得点計：1
保存完了
経過時間：238.42835807800293
全データ完了


In [8]:
pd.DataFrame(ALL_Datas).to_excel('エクセルファイル/全試合元データ/' + GameID + '修正要.xlsx', index = False)

In [6]:
Bat_info

[]

In [5]:
Inning_data

'4回裏'

In [7]:
Batt_Name

'R.ロドリゲス'